Ви аналітик даних у відділі Operational Excellence великого онлайн-маркетплейсу товарів. У складі маркетплейсу 2
сервіси:
- Платформа для роздрібного продажу.
- Платформа для гуртового продажу.

Після численних скарг про низьку якість роботи відділу підтримки, вам доручено проаналізувати, як можна покращити
рівень обслуговування клієнтів.

Вхідні дані:
- Кожен продукт має свою команду підтримки.
- Кожне завдання старанний агент служби підтримки може виконувати до 5 хвилин з момента взяття в роботу.
- З боку менеджменту є очікування, що в рамках кожного дня середній час від виникнення нового запиту користувача до відповіді має бути 15 хвилин. Також погано, якщо користувачі сервісу чекають на відповідь довше 45 хвилин.

Проаналізуйте дані про роботу служби підтримки за період та запропонуйте шляхи для підвищення якості
обслуговування.

Орієнтовний (але не вичерпний) приклад запитань для відповідей:
- Наскільки служба підтримки відповідає очікуванням менеджменту? Як вона працює зараз?
- На яких агентів варто звернути увагу задля підвищення якості обслуговування?
- Чи варто збільшити команду підтримки і наскільки?
- Як варто змінити графік служби підтримки?
- Чи є вигода від об'єднання команд?


**Опис даних:**

Таблиця events - дані про кожний запит до модерації. Містять в собі інформацію про сам запит та модератора, який його виконував.

- id_request - унікальний ідентифікатор запиту;
- moderator - унікальний ідентифікатор агента підтримки;
- team - параметр команди (retail / wholesale);
- request_time - час отримання запиту;
- start_time - час початку обробки кейсу;
- finish_time - час кінця обробки кейсу.

In [11]:
SELECT *
FROM 'events.csv'
LIMIT 5;

,moderator,id_request,request_time,start_time,finish_time,team
0,165,159660,2020-09-24 07:01:16+00:00,2020-10-01 16:00:31+00:00,2020-10-01 16:02:02+00:00,retail
1,178,160116,2020-09-24 22:32:15+00:00,2020-10-01 13:21:42+00:00,2020-10-01 13:21:43+00:00,retail
2,187,160178,2020-09-25 02:58:13+00:00,2020-10-02 11:37:16+00:00,2020-10-02 11:41:56+00:00,wholesale
3,178,160306,2020-09-25 09:44:12+00:00,2020-10-02 07:23:09+00:00,2020-10-02 07:25:33+00:00,retail
4,178,163042,2020-09-30 15:56:08+00:00,2020-10-02 07:37:33+00:00,2020-10-02 07:47:57+00:00,retail


## 1.Підготовка даних

In [34]:
-- Перевірка наявності всіх потрібних колонок та кількості записів
SELECT COUNT(*) FROM 'events.csv';

,count_star()
0,35617


In [22]:
-- Перевірка на пропущені значення
SELECT 
  COUNT(*) FILTER (WHERE id_request IS NULL) AS missing_id_request,
  COUNT(*) FILTER (WHERE moderator IS NULL) AS missing_moderator,
  COUNT(*) FILTER (WHERE team IS NULL) AS missing_team,
  COUNT(*) FILTER (WHERE request_time IS NULL) AS missing_request_time,
  COUNT(*) FILTER (WHERE start_time IS NULL) AS missing_start_time,
  COUNT(*) FILTER (WHERE finish_time IS NULL) AS missing_finish_time
FROM 'events.csv';

,missing_id_request,missing_moderator,missing_team,missing_request_time,missing_start_time,missing_finish_time
0,0,0,0,0,0,0


In [23]:
-- Перевірка діапазону дат
SELECT MIN(request_time) AS start_period, MAX(request_time) AS end_period FROM 'events.csv';

,start_period,end_period
0,2020-09-24 07:01:16+00:00,2020-12-31 22:11:17+00:00


### Результат перевірки данних
- Всього записів: 35 617
- Всі потрібні колонки на місці
- Пропущених значень немає
- Період даних: 24 вересня — 31 грудня 2020

## 2.Розрахунок ключових метрик

In [25]:
-- 2.1 Час до початку обробки (Response Time)
SELECT 
  id_request,
  moderator,
  team,
  EXTRACT(EPOCH FROM (start_time - request_time)) / 60 AS response_minutes
FROM 'events.csv'
LIMIT 5;

,id_request,moderator,team,response_minutes
0,159660,165,retail,10619.250000
1,160116,178,retail,9529.450000
2,160178,187,wholesale,10599.050000
3,160306,178,retail,9938.950000
4,163042,178,retail,2381.416667


,team,avg_response_minutes
0,retail,607.79
1,wholesale,3677.49
2,Total,2468.73


In [26]:
-- Середній Response Time по кожній команді та загалом
SELECT 
  team,
  ROUND(AVG(EXTRACT(EPOCH FROM (start_time - request_time)) / 60), 2) AS avg_response_minutes
FROM 'events.csv'
GROUP BY team
UNION ALL
SELECT 
  'Total' AS team,
  ROUND(AVG(EXTRACT(EPOCH FROM (start_time - request_time)) / 60), 2) AS avg_response_minutes
FROM 'events.csv';

,team,avg_response_minutes
0,retail,607.79
1,wholesale,3677.49
2,Total,2468.73


### Середній час до відповіді (Response Time)
- retail: 607.79 хв (~10 год)
- wholesale: 3677.49 хв (~61 год)
- всього: 2468.73 хв (~41 год)
- Очікування від менеджменту — до 15 хв.
  
#### Як висновок - обидві команди сильно перевищують очікуваний час до відповіді.

#### Що може бути причиною - або дуже мала кількість доступних агентів у команді wholesale, або неправильна організація черговості запитів.

In [28]:
-- 2.2 Час обробки запиту (Handling Time)
SELECT 
  id_request,
  moderator,
  EXTRACT(EPOCH FROM (finish_time - start_time)) / 60 AS handling_minutes
FROM 'events.csv'
LIMIT 5;

,id_request,moderator,handling_minutes
0,159660,165,1.516667
1,160116,178,0.016667
2,160178,187,4.666667
3,160306,178,2.400000
4,163042,178,10.400000


In [30]:
-- Середній Handling Time
SELECT 
  ROUND(AVG(EXTRACT(EPOCH FROM (finish_time - start_time)) / 60), 2) AS avg_handling_minutes
FROM 'events.csv';

,avg_handling_minutes
0,2.24


In [31]:
-- Кількість запитів > 5 хв
SELECT 
  COUNT(*) AS total,
  COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - start_time)) / 60 > 5) AS over_5_min,
  ROUND(100.0 * COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - start_time)) / 60 > 5) / COUNT(*), 2) AS percent_over_5
FROM 'events.csv';

,total,over_5_min,percent_over_5
0,35617,3802,10.67


### Середній час обробки (Handling Time)
- Середнє значення: 2.24 хв
- Запитів > 5 хв: 10.67%

#### Як висновок - переважна більшість запитів обробляється швидко, що відповідає очікуванням.

In [32]:
-- 2.3 Затримки понад 45 хвилин (від request до finish)
SELECT 
  COUNT(*) AS total_requests,
  COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) AS delayed_requests,
  ROUND(100.0 * COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) / COUNT(*), 2) AS percent_delayed
FROM 'events.csv';

,total_requests,delayed_requests,percent_delayed
0,35617,28232,79.27


### Загальні затримки > 45 хв
- Всього таких запитів: 79.27%
  
#### Як висновок - більшість користувачів чекають значно довше, ніж допустимі 45 хв — проблема в старті обробки, не в самій обробці.

## 3.Аналіз по агентам

In [8]:
-- Середні response та handling по кожному агенту
SELECT 
  moderator,
  ROUND(AVG(EXTRACT(EPOCH FROM (start_time - request_time)) / 60), 2) AS avg_response,
  ROUND(AVG(EXTRACT(EPOCH FROM (finish_time - start_time)) / 60), 2) AS avg_handling
FROM 'events.csv'
GROUP BY moderator
ORDER BY avg_response DESC
LIMIT 5;

,moderator,avg_response,avg_handling
0,126,5996.45,0.55
1,181,4692.15,2.80
2,180,4579.58,1.91
3,112,4382.39,2.28
4,187,4372.23,3.07


In [11]:
-- Агенти, що перевищують 45 хвилин
SELECT 
  moderator,
  COUNT(*) AS total_requests,
  COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) AS delayed,
  ROUND(100.0 * COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) / COUNT(*), 2) AS percent_delayed
FROM 'events.csv'
GROUP BY moderator
ORDER BY percent_delayed DESC
LIMIT 5;

,moderator,total_requests,delayed,percent_delayed
0,133,2,2,100.00
1,185,25,25,100.00
2,126,2,2,100.00
3,102,1321,1312,99.32
4,188,3746,3711,99.07


Бачимо, що частина агентів майже завжди працює із затримкою.

In [13]:
-- Агенти, які не встигають за 5 хвилин
SELECT 
  moderator,
  COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - start_time)) / 60 > 5) AS slow_cases,
  COUNT(*) AS total,
  ROUND(100.0 * COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - start_time)) / 60 > 5) / COUNT(*), 2) AS percent_slow
FROM 'events.csv'
GROUP BY moderator
ORDER BY percent_slow DESC
LIMIT 5;

,moderator,slow_cases,total,percent_slow
0,150,9,16,56.25
1,133,1,2,50.00
2,149,239,751,31.82
3,167,149,536,27.80
4,175,107,420,25.48


Агенти з найбільшим відсотком затримки потребують контролю або додаткового навчання.
Можлива відсутність SLA або контролю в wholesale.


## 4.Аналіз за командами

In [14]:
-- Порівняння response та handling time для retail та wholesale
SELECT 
  team,
  ROUND(AVG(EXTRACT(EPOCH FROM (start_time - request_time)) / 60), 2) AS avg_response,
  ROUND(AVG(EXTRACT(EPOCH FROM (finish_time - start_time)) / 60), 2) AS avg_handling
FROM 'events.csv'
GROUP BY team;

,team,avg_response,avg_handling
0,retail,607.79,1.98
1,wholesale,3677.49,2.42


,team,total_requests
0,retail,14025
1,wholesale,21592


In [15]:
-- Кількість запитів по командах
SELECT team, COUNT(*) AS total_requests FROM 'events.csv' GROUP BY team;

,team,total_requests
0,retail,14025
1,wholesale,21592


,team,over_45_min,total,percent_delayed
0,retail,7708,14025,54.96
1,wholesale,20524,21592,95.05


In [16]:
-- Затримки понад 45 хвилин по командах
SELECT 
  team,
  COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) AS over_45_min,
  COUNT(*) AS total,
  ROUND(100.0 * COUNT(*) FILTER (WHERE EXTRACT(EPOCH FROM (finish_time - request_time)) / 60 > 45) / COUNT(*), 2) AS percent_delayed
FROM 'events.csv'
GROUP BY team;

,team,over_45_min,total,percent_delayed
0,retail,7708,14025,54.96
1,wholesale,20524,21592,95.05


Місце для покращення є команда Wholesale — у них майже всі відповіді із значним запізненням. Wholesale обробляє на 50% більше кейсів, але при цьому набагато повільніше.

## 5.Аналіз по навантаженню

In [17]:
-- Кількість запитів на агента
SELECT 
  moderator,
  COUNT(*) AS total_requests
FROM 'events.csv'
GROUP BY moderator
ORDER BY total_requests DESC
LIMIT 5;

,moderator,total_requests
0,188,3746
1,112,1717
2,170,1686
3,181,1625
4,123,1452


In [35]:
-- Кількість запитів на агента по днях
SELECT 
  moderator,
  CAST(request_time AS DATE) AS day,
  COUNT(*) AS daily_requests
FROM 'events.csv'
GROUP BY moderator, CAST(request_time AS DATE)
ORDER BY moderator, day
LIMIT 5;

,moderator,day,daily_requests
0,102,2020-10-25 00:00:00+00:00,1
1,102,2020-11-16 00:00:00+00:00,15
2,102,2020-11-17 00:00:00+00:00,4
3,102,2020-11-20 00:00:00+00:00,25
4,102,2020-11-21 00:00:00+00:00,37


Топ 5 агентів обробили понад 10 000 кейсів — можливе перевантаження.

## 6.Аналіз по піковим годинам

,hour,total_requests
0,0,587
1,1,1507
2,2,1818
3,3,1910
4,4,1058
5,5,1450
6,6,2039
7,7,2572
8,8,3386
9,9,2940


In [19]:
-- Розподіл запитів по годинах доби
SELECT 
  EXTRACT(HOUR FROM request_time) AS hour,
  COUNT(*) AS total_requests
FROM 'events.csv'
GROUP BY hour
ORDER BY hour;

,hour,total_requests
0,0,587
1,1,1507
2,2,1818
3,3,1910
4,4,1058
5,5,1450
6,6,2039
7,7,2572
8,8,3386
9,9,2940


,weekday,total_requests
0,Sunday,5867
1,Saturday,5837
2,Friday,5818
3,Wednesday,5210
4,Thursday,5105
5,Monday,4382
6,Tuesday,3398


In [20]:
-- Розподіл по днях тижня
SELECT 
  STRFTIME(request_time, '%A') AS weekday,
  COUNT(*) AS total_requests
FROM 'events.csv'
GROUP BY weekday
ORDER BY total_requests DESC;

,weekday,total_requests
0,Sunday,5867
1,Saturday,5837
2,Friday,5818
3,Wednesday,5210
4,Thursday,5105
5,Monday,4382
6,Tuesday,3398


### По годинах
Найбільше запитів: 08:00 (3386), 09:00 (2940)

### По днях
Найбільше запитів: Неділя (5867), Субота (5837), Пʼятниця (5818)

#### Як висновок- пікові години — ранок, пікові дні — вихідні та пʼятниця. Варто посилити зміну у ці періоди.

## 7.Аналіз по годинам для 8- та 12-годинних змін

Навантаження для одного агента на зміну, виходячи з того, що:
- 1 запит = 5 хвилин (300 секунд) — згідно очікувань менеджменту
- 8 год × 60 хв / 5 хв = 96 запитів
- 12 год × 60 хв / 5 хв = 144 запити

,hour,total_requests,agents_needed_8h,agents_needed_12h
0,0,587,6.11,4.08
1,1,1507,15.70,10.47
2,2,1818,18.94,12.63
3,3,1910,19.90,13.26
4,4,1058,11.02,7.35
5,5,1450,15.10,10.07
6,6,2039,21.24,14.16
7,7,2572,26.79,17.86
8,8,3386,35.27,23.51
9,9,2940,30.63,20.42


In [69]:
-- Навантаження по годинах доби для 8/12 змін 
SELECT 
  EXTRACT(HOUR FROM request_time) AS hour,
  COUNT(*) AS total_requests,
  ROUND(COUNT(*) / 96.0, 2) AS agents_needed_8h,   
  ROUND(COUNT(*) / 144.0, 2) AS agents_needed_12h  
FROM 'events.csv'
GROUP BY hour
ORDER BY total_requests;

,hour,total_requests,agents_needed_8h,agents_needed_12h
0,23,352,3.67,2.44
1,22,354,3.69,2.46
2,21,449,4.68,3.12
3,0,587,6.11,4.08
4,20,663,6.91,4.60
5,18,697,7.26,4.84
6,19,721,7.51,5.01
7,17,772,8.04,5.36
8,16,989,10.30,6.87
9,4,1058,11.02,7.35


,weekday,total_requests,agents_needed_8h,agents_needed_12h
0,0,5867,61.11,40.74
1,1,4382,45.65,30.43
2,2,3398,35.40,23.60
3,3,5210,54.27,36.18
4,4,5105,53.18,35.45
5,5,5818,60.60,40.40
6,6,5837,60.80,40.53


In [71]:
-- Навантаження по днях тижня  для 8/12 змін (0 = Пн, 6 = Нд)
SELECT 
  EXTRACT(DOW FROM request_time) AS weekday,  
  COUNT(*) AS total_requests,
  ROUND(COUNT(*) / 96.0, 2) AS agents_needed_8h,
  ROUND(COUNT(*) / 144.0, 2) AS agents_needed_12h
FROM 'events.csv'
GROUP BY weekday
ORDER BY total_requests DESC;

,weekday,total_requests,agents_needed_8h,agents_needed_12h
0,0,5867,61.11,40.74
1,6,5837,60.80,40.53
2,5,5818,60.60,40.40
3,3,5210,54.27,36.18
4,4,5105,53.18,35.45
5,1,4382,45.65,30.43
6,2,3398,35.40,23.60


### Висновки по годинам:
- Вранці (08:00–09:00) потрібність до 35 агентів  для 8 год або 24  для 12 год.
- Уночі (02:00–03:00) потрібність в середньому 13–19 агентів.
- 12-годинна зміна дозволить зменшити потребу в персоналі на 30–35%.

### Висновки по дням тижня:
- Найбільше навантаження у вихідні (Сб, Нд): що призводить у потребі понад 60 агентів при 8-годинному графіку.
- Перехід на 12-годинну зміну дозволить зменшити потребу до 40.

## Загальний висновок
- Основна проблема — надто довгий час до відповіді, а не сама обробка.
- Команда Wholesale має критичне перевантаження.
- Частина агентів не справляється, хоча обробка коротка.
- Слід оптимізувати графіки, впровадити змінний підхід і зменшити навантаження на пік.

## Рекомендації

1. Обʼєднання команд retail і wholesale — неефективне, бо це може лише ускладнити ситуацію
2. Потрібно терміново оптимізувати організацію роботи команди wholesale (гнучкий графік / чергування, більше агентів, автоматичне пріоритезування запитів)
3. Слід впровадити SLA-контроль, особливо для кейсів >15 і >45 хв.
4. Деякі агенти потребують індивідуального аудиту, тренингу або перерозподілу навантаження: 102, 188, 150, 149
5. По змінам в графіку потрібно  ввести гнучкий графік (плаваючі зміни), додати нічні зміни (частково автоматизоване), використати 12-годинні зміни у вихідні